In [ ]:
from tensorflow.keras.datasets import mnist
import numpy as np

# Convert images to frequency domain using 2D FFT
def preprocess_images(images):
    processed_images = []
    for img in images:
        freq_img = (np.fft.fft2(img))
        #taking only the real part of the frequency image
        freq_img = np.real(freq_img)
        processed_images.append(freq_img)
    return np.array(processed_images)

# Load the dataset
(X_train, _), (_, _) = mnist.load_data()

# Rescale -1 to 1 and preprocess images
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)
X_train = preprocess_images(X_train)



In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization, LeakyReLU, Lambda
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers.legacy import Adam
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
class GAN():
    def __init__(self, img_rows, img_cols, channels, latent_dim):
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = latent_dim

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(28 * 28, activation='tanh'))  # Output shape: (None, 28*28)
        model.add(Reshape((28, 28, 1)))  # Reshape to (None, 28, 28, 1)

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)


    def build_discriminator(self):
        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))

        # FFT layer
        model.add(Lambda(lambda x: tf.signal.fft2d(tf.cast(x, tf.complex64))))
        model.add(Lambda(lambda x: tf.abs(x)))  # Convert complex values to magnitude

        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        #plot the model

        freq_img = Input(shape=self.img_shape)
        validity = model(freq_img)

        return Model(freq_img, validity)

    def train(self, epochs, batch_size, save_interval):
        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # If at save interval, print and save generated image samples
            if epoch % save_interval == 0:
                print(f"Epoch {epoch}: [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]:.2f}%] [G loss: {g_loss}]")

                # Save generated images
                self.save_images(epoch)

    def save_images(self, epoch):
        r, c = 5, 5  # Generate 5x5 grid of images
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, 2 * c)  # Double the number of columns for real and imaginary parts
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, 2 * j].imshow(gen_imgs[cnt, :, :, 0].real, cmap='gray')
                axs[i, 2 * j].axis('off')
                axs[i, 2 * j + 1].imshow(gen_imgs[cnt, :, :, 0].imag, cmap='gray')
                axs[i, 2 * j + 1].axis('off')
                cnt += 1
        fig.savefig(f"fft_gans/mnist_{epoch}.png")
        plt.close()



# Create an instance of the GAN class
gan = GAN(img_rows=28, img_cols=28, channels=1, latent_dim=100)
#Train Gan
gan.train(epochs=1000, batch_size=128, save_interval=10)

KeyboardInterrupt: 

In [ ]:
img_path = "/home/kalyan/Pictures/screenshots/14-04-2023 11:51:38.png"
plt.imshow(plt.imread(img_path))
#take the image and convert it to frequency domain
img = plt.imread(img_path)

In [ ]:

#take fft of the image
img_fft = (np.fft.fft2(img))
#take the real part of the image
img_real = img_fft.real
#plot the image
plt.imshow(img_real)
plt.show()


In [ ]:
#save model weights
gan.generator.save_weights('gan_imagesss/generator_weights.h5')
gan.discriminator.save_weights('gan_imagesss/discriminator_weights.h5')
